### Nombre: Keshava Tonathiu Sanchez Barbosa (418127029)
### Materia: Machine Learning
### Semestre: 4°

## **Tarea: RF** (evaluada en la categoría _Otras evaluaciones_)

In [1]:
# Common imports
import numpy as np

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

### a) Entrenar y ajustar un árbol de decisión para el conjunto de datos _Moon_

In [2]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

### b) Dividir en conjunto de entrenamiento y de prueba utilizando ```train_test_split()```
#### **Importante**: _Todas las pruebas_ se realizarán prediciendo sobre ```X_test``` y comparando dichas predicciones contra ```y_test```

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### c) Utilizar la búsqueda en malla con validación cruzada (con ayuda de la clase ```GridSearchCV```) para encontrar valores aceptables para los metaparámetros del ```DecisionTreeClassifier```:

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1, verbose=1, cv=3)

grid_search_cv.fit(X_train, y_train)

Fitting 3 folds for each of 294 candidates, totalling 882 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 882 out of 882 | elapsed:    2.4s finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], 'min_samples_split': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       s

### d) Mostrar el mejor estimador encontrado:

In [5]:
grid_search_cv.best_estimator_

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=17,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [6]:
grid_search_cv.best_params_

{'max_leaf_nodes': 17, 'min_samples_split': 2}

### e) Por omisión, ```GridSearchCV``` entrena el mejor modelo encontrado con todo el conjunto de datos (entrenamiento + prueba) al terminar la búsqueda (esto se puede cambiar pasándole a ```GridSearchCV``` el parámetro ```refit=False```). Con lo anterior, evaluamos la exactitud del modelo directamente:

In [7]:
from sklearn.metrics import accuracy_score

y_pred = grid_search_cv.predict(X_test)
accuracy_score(y_test, y_pred)

0.8695

***
## **Trabajo a realizar**
***

### En este trabajo haremos la comparación de los resultados obtenidos al realizar GridSearchCV con la predicción que se logra obtener de entrenar arboles con los mejores parametros encontrados y posteriormente encontrando la predicción mas común.

### 1.- Generar 1,000 subconjuntos del conjunto de entrenamiento ```X_train```, cada uno con 100 instancias seleccionadas aleatoriamente. La selección es con reemplazo, es decir, no _sacan_ de ```X_train``` las instancias que vayan seleccionando (y no olviden copiar también de ```y_train``` las etiquetas correspondientes). Pueden utilizar la clase ```ShuffleSplit``` de Scikit-Learn para esto.
#### **Nota**.- No hay subconjuntos de validación aquí, todas las pruebas se harán contra ```X_test``` y ```y_test```, como se indicó más arriba.

se hace uso de ShuffleSplit para obtener los conjuntos de indices para generar nuestros arboles

In [8]:
from sklearn.model_selection import ShuffleSplit

random_split = ShuffleSplit(n_splits=1000, test_size=0, train_size=100, random_state=42)
random_split.get_n_splits(X_train)

1000

se crea una lista de "arboles" donde almacenamos cada uno de los subconjuntos de indices que genero nuestro ShuffleSplit

In [9]:
arboles = []
for train in random_split.split(X_train, y = None, groups = None):
    arboles.append(train[0])
print(len(arboles))

1000


### 2.- Entrenar un árbol de decisión en cada subconjunto de 100 instancias, usando los mejores valores de los metaparámetros encontrados por la búsqueda en malla, es decir, el estimador devuelto por ```grid_search_cv.best_estimator_```.

Creamos los arboles y sus conjuntos de soluciones a partir de un ciclo for,  posteriormente haciendo uso del best_estimator_ y best_params_ entrenamos cada uno de los arboles creados

In [10]:
from sklearn.tree import DecisionTreeClassifier
arbol_shuffle = []
arbol_entrenado = []
y_train_2_electricbogaloo = []
for i in range(len(arboles)):
    arbol_actual = []
    y_actual = []
    for j in range(100):
        indice = arboles[i][j]
        arbol_actual.append(X_train[indice])
        y_actual.append(y_train[indice])
    arbol_shuffle.append(arbol_actual)
    y_train_2_electricbogaloo.append(y_actual)
        
for i in range(len(arboles)):
    tree_sk = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                max_features=None,
                min_impurity_decrease=0.0, min_impurity_split=None,
                min_samples_leaf=1,
                min_weight_fraction_leaf=0.0, presort=False, random_state=42,
                splitter='best', max_leaf_nodes = 17, min_samples_split = 2)
    arbol_entrenado.append(tree_sk.fit(arbol_shuffle[i], y_train_2_electricbogaloo[i]))

### 4.- Llega la magia: generar las predicciones de cada uno de los 1,000 árboles de decisión para cada instancia en el conjunto de prueba.

### 3.- Evaluar cada uno de estos 1,000 árboles de decisión sobre el conjunto de prueba (```X_test``` y ```y_test```). Sólo deben calcular los 1,000 ```accuracy_score``` y promediarlos. Dado que los árboles se entrenarán con conjuntos de datos más pequeños que con el árbol de decisión determinado por la búsqueda en malla, la exactitud promedio de estos mil árboles será muy probablemente más baja (resultará un 80% de exactitud, aproximadamente):

In [16]:
pred = []
scores = []
avg = 0
for j in range(len(arbol_entrenado)):  
    y_predict = arbol_entrenado[j].predict(X_test)
    pred.append(y_predict)
    score = accuracy_score(y_test, y_predict)
    avg += score
    scores.append(score)
    
avg /= 1000
print(avg)

0.8062925000000005


El promedio de las predicciones termina siendo menor que nuestra predicción por GridSearch (0.8695)

### 5.- Conservar solamente la predicción más frecuente (se puede utilizar la función ```mode()``` de SciPy para esto). Esto corresponde a una predicción por mayoría de votos sobre el conjunto de prueba.

In [30]:
from scipy import stats
moda = stats.mode(pred)

### 6.- Evaluar las predicciones conservadas ```y_pred_majority_votes``` contra el conjunto de prueba ```y_test```: se debe obtener un  ```accuracy_score``` ligeramente más alto que el del árbol de decisión de la búsqueda en malla (entre el 0.5 y el 1.5%).

In [31]:
moda_score = accuracy_score(y_test, moda[0][0])
print(moda_score)

0.875


Sin embargo, si obtenemos la moda de los resultados obtenidos podemos observar que este resultado es mejor que el obtenido por GridSearch (0.8695)

### Conclusion

Podemos observar que existe una mejora entre nuestro resultado mas comun generado por los arboles (0.875) y el resultado generado por GridSearch (0.8695), parte de esto es por el algoritmo que se realiza con el entrenamiento de los arboles, y otra parte puede ser dada por los metaparametros obtenidos, lo cual mejora el accuracy de nuestro resultado.